# NOT SELECTED DUE TOLONG EXECUTION TIME -> 24h plus 

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import json
from tqdm.auto import tqdm
import torch
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

print(f"Torch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MPS available: {torch.backends.mps.is_available()}")

Torch version: 2.9.0
CUDA available: False
MPS available: True


In [7]:
# Configuration
BASE_DIR = Path('/Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment')
INPUT_DIR = BASE_DIR / 'data' / '01_corpus' / '03_qa' / 'reddit'
DATA_DIR = BASE_DIR / 'data' / '02_topics' / '03_gold' / 'reddit'
OUTPUT_DIR = DATA_DIR
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Model configuration
MODEL_NAME = 'facebook/bart-large-mnli'
BATCH_SIZE = 16  # Process 16 documents at a time
USE_GPU = True   # Use MPS (Apple Silicon) or CUDA if available

# Determine device
if USE_GPU:
    if torch.cuda.is_available():
        device = 0  # CUDA device
        print("Using CUDA GPU")
    elif torch.backends.mps.is_available():
        device = 0  # MPS will be handled by transformers
        print("Using Apple Silicon MPS")
    else:
        device = -1  # CPU
        print("No GPU available, using CPU")
else:
    device = -1
    print("Using CPU (GPU disabled)")

print(f"\nConfiguration:")
print(f"  Data directory: {DATA_DIR}")
print(f"  Model: {MODEL_NAME}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Device: {device}")

Using Apple Silicon MPS

Configuration:
  Data directory: /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment/data/02_topics/03_gold/reddit
  Model: facebook/bart-large-mnli
  Batch size: 16
  Device: 0


## Define Political Topic Taxonomy

20 political topics adapted from the Comparative Agendas Project (CAP) framework.

In [8]:
# Define 20 political topics with natural language descriptions
# Based on Comparative Agendas Project (CAP) codebook

POLITICAL_TOPICS = {
    0: {
        'label': 'Elections & Voting',
        'description': 'Electoral processes, campaigns, voting rights, electoral reform, and voter registration'
    },
    1: {
        'label': 'Presidential Politics',
        'description': 'Presidential actions, administration policies, executive orders, and White House activities'
    },
    2: {
        'label': 'Congress & Legislation',
        'description': 'Congressional activities, legislative processes, lawmakers, and parliamentary procedures'
    },
    3: {
        'label': 'Healthcare Policy',
        'description': 'Health insurance, Affordable Care Act, medical costs, public health, and healthcare reform'
    },
    4: {
        'label': 'Immigration & Borders',
        'description': 'Immigration policy, border security, refugee issues, deportation, and asylum'
    },
    5: {
        'label': 'Economy & Employment',
        'description': 'Jobs, unemployment, wages, labor issues, economic growth, and workplace regulations'
    },
    6: {
        'label': 'Budget & Taxation',
        'description': 'Federal budget, taxes, government spending, deficits, and fiscal policy'
    },
    7: {
        'label': 'Education Policy',
        'description': 'Schools, universities, student loans, education reform, and academic issues'
    },
    8: {
        'label': 'Criminal Justice',
        'description': 'Crime, policing, prisons, law enforcement, justice system, and criminal sentencing'
    },
    9: {
        'label': 'Gun Rights & Control',
        'description': 'Second Amendment, gun violence, firearms regulation, and gun ownership'
    },
    10: {
        'label': 'Environment & Climate',
        'description': 'Climate change, EPA regulations, pollution, conservation, and environmental protection'
    },
    11: {
        'label': 'Energy Policy',
        'description': 'Oil, gas, renewable energy, energy independence, and energy regulations'
    },
    12: {
        'label': 'Foreign Policy & Diplomacy',
        'description': 'International relations, diplomacy, global issues, and foreign affairs'
    },
    13: {
        'label': 'Defense & Military',
        'description': 'Military operations, veterans, defense spending, armed forces, and national security'
    },
    14: {
        'label': 'Trade Policy',
        'description': 'International trade, tariffs, trade agreements, imports, and exports'
    },
    15: {
        'label': 'Social Issues',
        'description': 'Abortion, LGBTQ rights, religious freedom, family values, and moral issues'
    },
    16: {
        'label': 'Civil Rights & Discrimination',
        'description': 'Racial justice, discrimination, equality issues, and civil rights movements'
    },
    17: {
        'label': 'Media & Free Speech',
        'description': 'Press freedom, censorship, media bias, First Amendment, and journalism'
    },
    18: {
        'label': 'Technology & Privacy',
        'description': 'Tech regulation, surveillance, data privacy, cybersecurity, and digital rights'
    },
    19: {
        'label': 'Infrastructure',
        'description': 'Roads, bridges, public transit, infrastructure investment, and transportation'
    }
}

# Extract labels and descriptions for classification
topic_labels = [POLITICAL_TOPICS[i]['label'] for i in range(20)]
topic_descriptions = [POLITICAL_TOPICS[i]['description'] for i in range(20)]

print("\n=== Political Topic Taxonomy (20 Topics) ===")
print("\nSource: Comparative Agendas Project (CAP)\n")
for topic_id, topic_info in POLITICAL_TOPICS.items():
    print(f"{topic_id:2d}. {topic_info['label']}")
    print(f"    {topic_info['description']}")
    print()


=== Political Topic Taxonomy (20 Topics) ===

Source: Comparative Agendas Project (CAP)

 0. Elections & Voting
    Electoral processes, campaigns, voting rights, electoral reform, and voter registration

 1. Presidential Politics
    Presidential actions, administration policies, executive orders, and White House activities

 2. Congress & Legislation
    Congressional activities, legislative processes, lawmakers, and parliamentary procedures

 3. Healthcare Policy
    Health insurance, Affordable Care Act, medical costs, public health, and healthcare reform

 4. Immigration & Borders
    Immigration policy, border security, refugee issues, deportation, and asylum

 5. Economy & Employment
    Jobs, unemployment, wages, labor issues, economic growth, and workplace regulations

 6. Budget & Taxation
    Federal budget, taxes, government spending, deficits, and fiscal policy

 7. Education Policy
    Schools, universities, student loans, education reform, and academic issues

 8. Crimi

## Load Data

In [10]:
# Load thread pseudo-documents
input_file = INPUT_DIR / 'thread_pseudodocs.parquet'

print(f"Loading data from: {input_file}")
thread_docs = pd.read_parquet(input_file)

print(f"\nData loaded:")
print(f"  Total documents: {len(thread_docs):,}")
print(f"  Date range: {thread_docs['created_utc'].min()} to {thread_docs['created_utc'].max()}")
print(f"  Columns: {list(thread_docs.columns)}")
print(f"\nSample document:")
print(thread_docs['pseudodoc_text'].iloc[0][:500] + '...')

Loading data from: /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment/data/01_corpus/03_qa/reddit/thread_pseudodocs.parquet

Data loaded:
  Total documents: 433,973
  Date range: 1472688024 to 1477954796
  Columns: ['submission_id', 'title', 'selftext', 'n_comments', 'pseudodoc_text', 'pseudodoc_length', 'pseudodoc_tokens_approx', 'title_length', 'selftext_length', 'created_utc', 'subreddit', 'subreddit_id']

Sample document:
When is the speech?  I thought it was at 7 pm? like right now!...


## Initialize Zero-Shot Classifier

In [11]:
# Initialize zero-shot classification pipeline
print(f"Loading model: {MODEL_NAME}")
print("This may take a few minutes on first run...\n")

classifier = pipeline(
    "zero-shot-classification",
    model=MODEL_NAME,
    device=device
)

print("✓ Model loaded successfully")

# Test on a single document
print("\nTesting classifier on sample document...\n")
test_text = thread_docs['pseudodoc_text'].iloc[0][:512]  # Truncate to 512 tokens

test_result = classifier(
    test_text,
    candidate_labels=topic_labels,
    multi_label=False  # Single topic per document
)

print(f"Sample classification:")
print(f"  Top topic: {test_result['labels'][0]}")
print(f"  Confidence: {test_result['scores'][0]:.3f}")
print(f"\n  Top 3 predictions:")
for label, score in zip(test_result['labels'][:3], test_result['scores'][:3]):
    print(f"    {label}: {score:.3f}")

Loading model: facebook/bart-large-mnli
This may take a few minutes on first run...



Device set to use mps:0


✓ Model loaded successfully

Testing classifier on sample document...

Sample classification:
  Top topic: Media & Free Speech
  Confidence: 0.191

  Top 3 predictions:
    Media & Free Speech: 0.191
    Infrastructure: 0.087
    Defense & Military: 0.062


## Classify All Documents

Process documents in batches for efficiency. This may take 30-60 minutes for ~430k documents.

In [12]:
# Prepare documents for classification
# Truncate to max 512 tokens to avoid model limits
documents = thread_docs['pseudodoc_text'].tolist()
documents_truncated = [doc[:2048] for doc in documents]  # Approx 512 tokens

print(f"Classifying {len(documents_truncated):,} documents...")
print(f"Batch size: {BATCH_SIZE}")
print(f"Estimated time: ~{len(documents_truncated) // (BATCH_SIZE * 10)}-{len(documents_truncated) // (BATCH_SIZE * 5)} minutes\n")

# Initialize storage for results
topic_ids = []
topic_labels_assigned = []
topic_confidences = []
all_scores = []  # Store full probability distribution

# Process in batches with progress bar
start_time = datetime.now()

for i in tqdm(range(0, len(documents_truncated), BATCH_SIZE), desc="Classifying"):
    batch = documents_truncated[i:i + BATCH_SIZE]
    
    # Classify batch
    results = classifier(
        batch,
        candidate_labels=topic_labels,
        multi_label=False
    )
    
    # Handle single document vs batch results
    if not isinstance(results, list):
        results = [results]
    
    # Extract results
    for result in results:
        # Get top prediction
        top_label = result['labels'][0]
        top_score = result['scores'][0]
        
        # Find topic ID from label
        topic_id = topic_labels.index(top_label)
        
        topic_ids.append(topic_id)
        topic_labels_assigned.append(top_label)
        topic_confidences.append(top_score)
        
        # Store full probability distribution (aligned with topic IDs 0-19)
        score_dict = {label: score for label, score in zip(result['labels'], result['scores'])}
        aligned_scores = [score_dict[label] for label in topic_labels]
        all_scores.append(aligned_scores)

end_time = datetime.now()
duration = (end_time - start_time).total_seconds() / 60

print(f"\n✓ Classification complete!")
print(f"  Duration: {duration:.1f} minutes")
print(f"  Throughput: {len(documents_truncated) / duration:.0f} documents/minute")

Classifying 433,973 documents...
Batch size: 16
Estimated time: ~2712-5424 minutes



Classifying:   0%|          | 0/27124 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Analyze Results

In [ ]:
# Add results to dataframe
thread_docs['supervised_topic_id'] = topic_ids
thread_docs['supervised_topic_label'] = topic_labels_assigned
thread_docs['supervised_topic_confidence'] = topic_confidences

# Add full probability distribution (optional, for analysis)
for i in range(20):
    thread_docs[f'topic_{i}_prob'] = [scores[i] for scores in all_scores]

print("\n=== Classification Results ===")
print(f"\nTotal documents classified: {len(thread_docs):,}")
print(f"\nTopic distribution:")
topic_dist = thread_docs['supervised_topic_label'].value_counts().sort_index()
for label, count in topic_dist.items():
    pct = count / len(thread_docs) * 100
    print(f"  {label:30s}: {count:7,} ({pct:5.2f}%)")

print(f"\nConfidence statistics:")
print(f"  Mean confidence: {thread_docs['supervised_topic_confidence'].mean():.3f}")
print(f"  Median confidence: {thread_docs['supervised_topic_confidence'].median():.3f}")
print(f"  Min confidence: {thread_docs['supervised_topic_confidence'].min():.3f}")
print(f"  Max confidence: {thread_docs['supervised_topic_confidence'].max():.3f}")

# Low confidence documents (potential misclassifications)
low_conf_threshold = 0.3
low_conf = thread_docs[thread_docs['supervised_topic_confidence'] < low_conf_threshold]
print(f"\nLow confidence documents (<{low_conf_threshold}): {len(low_conf):,} ({len(low_conf)/len(thread_docs)*100:.2f}%)")

## Visualize Topic Distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

# Topic distribution
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# 1. Topic counts
topic_counts = thread_docs['supervised_topic_id'].value_counts().sort_index()
ax = axes[0]
topic_count_labels = [POLITICAL_TOPICS[i]['label'] for i in topic_counts.index]
ax.bar(range(len(topic_counts)), topic_counts.values, color='steelblue', alpha=0.7)
ax.set_xticks(range(len(topic_counts)))
ax.set_xticklabels(topic_count_labels, rotation=45, ha='right')
ax.set_ylabel('Document Count')
ax.set_title('Document Distribution Across Political Topics')
ax.grid(axis='y', alpha=0.3)

# 2. Confidence distribution by topic
ax = axes[1]
topic_conf = thread_docs.groupby('supervised_topic_id')['supervised_topic_confidence'].mean().sort_index()
topic_conf_labels = [POLITICAL_TOPICS[i]['label'] for i in topic_conf.index]
ax.bar(range(len(topic_conf)), topic_conf.values, color='coral', alpha=0.7)
ax.set_xticks(range(len(topic_conf)))
ax.set_xticklabels(topic_conf_labels, rotation=45, ha='right')
ax.set_ylabel('Mean Confidence')
ax.set_ylim(0, 1)
ax.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Threshold')
ax.set_title('Mean Classification Confidence by Topic')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'supervised_topic_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualization saved")

## Temporal Analysis

In [ ]:
# Temporal trends
thread_docs['date'] = pd.to_datetime(thread_docs['date_temp'])
temporal_df = thread_docs.groupby(['date', 'supervised_topic_label']).size().reset_index(name='count')

# Top 5 topics over time
top_topics = thread_docs['supervised_topic_label'].value_counts().head(5).index
temporal_top = temporal_df[temporal_df['supervised_topic_label'].isin(top_topics)]

fig, ax = plt.subplots(figsize=(14, 6))
for topic in top_topics:
    topic_data = temporal_top[temporal_top['supervised_topic_label'] == topic]
    ax.plot(topic_data['date'], topic_data['count'], label=topic, marker='o', markersize=3, alpha=0.7)

ax.set_xlabel('Date')
ax.set_ylabel('Document Count')
ax.set_title('Top 5 Topics Over Time (Sep-Oct 2016)')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'supervised_temporal_trends.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Temporal analysis complete")

## Save Results

In [ ]:
# Save main results (without full probability distribution)
output_columns = [
    'thread_id', 'date_temp', 'pseudodoc_text',
    'supervised_topic_id', 'supervised_topic_label', 'supervised_topic_confidence'
]

output_file = OUTPUT_DIR / 'thread_pseudodocs_with_supervised_topics.parquet'
thread_docs[output_columns].to_parquet(output_file, index=False)

print(f"✓ Main results saved to: {output_file}")
print(f"  Shape: {thread_docs[output_columns].shape}")

# Save full results with probability distributions (for advanced analysis)
output_file_full = OUTPUT_DIR / 'thread_pseudodocs_with_supervised_topics_full.parquet'
thread_docs.to_parquet(output_file_full, index=False)

print(f"✓ Full results (with probabilities) saved to: {output_file_full}")
print(f"  Shape: {thread_docs.shape}")

## Save Metadata

In [ ]:
# Save metadata
metadata = {
    'method': 'zero_shot_classification',
    'model': MODEL_NAME,
    'taxonomy': 'Comparative Agendas Project (CAP)',
    'taxonomy_citation': 'Baumgartner, F. R., Jones, B. D., & Wilkerson, J. (2011). Comparative Studies of Policy Agendas. Journal of European Public Policy, 18(5), 639-653.',
    'num_topics': 20,
    'topics': POLITICAL_TOPICS,
    'num_documents': len(thread_docs),
    'date_range': {
        'start': str(thread_docs['date_temp'].min()),
        'end': str(thread_docs['date_temp'].max())
    },
    'classification_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'duration_minutes': duration,
    'device': 'MPS' if device == 0 and torch.backends.mps.is_available() else 'CUDA' if device >= 0 else 'CPU',
    'batch_size': BATCH_SIZE,
    'confidence_stats': {
        'mean': float(thread_docs['supervised_topic_confidence'].mean()),
        'median': float(thread_docs['supervised_topic_confidence'].median()),
        'min': float(thread_docs['supervised_topic_confidence'].min()),
        'max': float(thread_docs['supervised_topic_confidence'].max()),
        'low_confidence_count': int(len(low_conf)),
        'low_confidence_threshold': low_conf_threshold
    },
    'topic_distribution': {
        topic_label: int(count) 
        for topic_label, count in thread_docs['supervised_topic_label'].value_counts().items()
    }
}

metadata_file = OUTPUT_DIR / 'supervised_classification_metadata.json'
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✓ Metadata saved to: {metadata_file}")

## Summary

In [ ]:
print("\n" + "="*80)
print("SUPERVISED TOPIC CLASSIFICATION - SUMMARY")
print("="*80)

print(f"\n📊 Dataset:")
print(f"  Documents classified: {len(thread_docs):,}")
print(f"  Date range: {thread_docs['date_temp'].min()} to {thread_docs['date_temp'].max()}")

print(f"\n🎯 Method:")
print(f"  Model: {MODEL_NAME}")
print(f"  Taxonomy: Comparative Agendas Project (CAP)")
print(f"  Number of topics: 20")

print(f"\n⚡ Performance:")
print(f"  Duration: {duration:.1f} minutes")
print(f"  Throughput: {len(thread_docs) / duration:.0f} documents/minute")
print(f"  Device: {metadata['device']}")

print(f"\n📈 Results:")
print(f"  Mean confidence: {thread_docs['supervised_topic_confidence'].mean():.3f}")
print(f"  Median confidence: {thread_docs['supervised_topic_confidence'].median():.3f}")
print(f"  Low confidence (<{low_conf_threshold}): {len(low_conf):,} ({len(low_conf)/len(thread_docs)*100:.2f}%)")

print(f"\n🏆 Top 5 Topics:")
for i, (topic, count) in enumerate(thread_docs['supervised_topic_label'].value_counts().head(5).items(), 1):
    pct = count / len(thread_docs) * 100
    print(f"  {i}. {topic:30s}: {count:7,} ({pct:5.2f}%)")

print(f"\n💾 Output Files:")
print(f"  Main results: {output_file.name}")
print(f"  Full results: {output_file_full.name}")
print(f"  Metadata: {metadata_file.name}")
print(f"  Visualizations: supervised_topic_distribution.png, supervised_temporal_trends.png")

print("\n✓ Classification complete!")
print("\nNext steps:")
print("  1. Run notebook 16b for quality assessment")
print("  2. Compare with STM/NMF (16 passing) and BERTopic (15 passing)")
print("  3. Validate low-confidence documents")
print("="*80)